In [355]:
import selenium
import os
import io
import base64
import requests
from PIL import Image
import time
import random
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [341]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 

In [366]:
try:
    requests.get('http://www.some-site.com/', timeout = 10)
except:
    print('yu fucked up boi')

yu fucked up boi


In [342]:
DRIVER_PATH = '/Users/khaira/Documents/chromedriver'
extension_path = './view_image_extension.crx'

In [343]:
#superman, spiderman, batman, hulk

In [344]:
options = webdriver.ChromeOptions()
options.add_extension('./view_image_extension.crx')

In [328]:
wd = webdriver.Chrome(executable_path=DRIVER_PATH, options= options)

In [329]:
go_to_image_page(wd, "batman")

In [333]:
divContainingImages = wd.find_element_by_id('islrg')

In [334]:
desiredImgsScrape = 350 

In [153]:
# scroll down the page until there are enough images loaded on the page and we can begin scraping!
while len(divContainingImages.find_elements_by_class_name('isv-r.PNCib.MSM1fd')) < desiredImgsScrape:
    print(len(divContainingImages.find_elements_by_class_name('isv-r.PNCib.MSM1fd')))
    scroll_to_bottom() 
print(len(divContainingImages.find_elements_by_class_name('isv-r.PNCib.MSM1fd')))

48
100
200
300
400


In [345]:
def scroll_to_bottom():
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [346]:
def go_to_image_page(wd, query):
    base_url = 'https://www.google.com/search?tbm=isch&q='
    wd.get(base_url + query)

In [354]:
class anchor_tag_has_http:
  """An expectation for checking that the view image tab extension is finalized loading and has the 
  appropriate anchor tag. 

  locator - used to find the element
  returns the WebElement once the http has finished loading 
  """
  def __init__(self, locator):
    self.locator = locator
    self.http = "http"

  def __call__(self, driver):
    element = driver.find_element_by_class_name(self.locator)
    if self.http in element.get_attribute("href"):
        return element
    else:
        return False


In [353]:
try:   
    WebDriverWait(wd, 1).until(anchor_tag_has_http('ZsbmCf.vi_ext_addon'))
except TimeoutException as ex:
    try:  
        WebDriverWait(wd, 1).until(anchor_tag_has_http('ZsbmCf.vi_ext_addon'))
    except:
        print('o well we tried')

o well we tried


In [332]:
count = 0
for box in boxesContainingImages:
    count += 1
    box.click()
    element = None
    # give the http 14 seconds to load in for the extension, if it doesn't load in this max
    # amount of time, then we assume its a base64 encoded image and continue 
    try:
        WebDriverWait(wd, 1).until(anchor_tag_has_http('ZsbmCf.vi_ext_addon'))
        actualImage = get_actual_image()
    # except:
    #     actualImage = get_actual_image()
    finally:
        print(count)
        box.click()
        if count == 12:
            break

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=56653): Max retries exceeded with url: /session/c01c0c564f37a2e7467fe066048f9c89/element/1f9f6a16-5e4f-4b9a-8b22-7110f72162a5/click (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10d95be80>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [314]:
def image_scraper(queries, numImagesDesired, driver_path, view_extension_crx_path):
    """
    This function scrapes full-sized images when given an input list containing the terms for 
    which images should be scraped. 

    Args:
        queries (list of str): List containing the terms for which images should be scraped 

        numImagesDesired (int): Integer representing the number of images to scrape for every
            query in the queries

        driver_path (str): String representing the path to the chrome driver to be used 
            with Selenium.

        view_extension_crx_path (str): String representing the path to the view extension .crx file

    Returns:
        None. A new folder is made for each query string, and the images associated with the query
            are saved inside of that folder. 

    """     
    base_url = 'https://www.google.com/search?tbm=isch&q='
    options = webdriver.ChromeOptions()
    options.add_extension(view_extension_crx_path) 
    wd = webdriver.Chrome(executable_path=driver_path, options= options)
    for query in queries:
        # go to the webpage containing all images
        wd.get(base_url + query)
        # initalize the directory that will hold all the images
        try:
            os.mkdir(os.path.abspath('') + "/"+query) 
        except:
            pass 
        divContainingImages = wd.find_element_by_id('islrg')
        extract_images(divContainingImages, query, wd, numImagesDesired)

In [315]:
def extract_images(divContainingImages, query, wd, numImagesDesired):
    # function for scrolling to bottom in case there aren't enough images loaded on the page 
    def scroll_to_bottom():
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    count = 0
    # scroll down the page until there are enough images loaded on the page and we can begin scraping!
    while len(divContainingImages.find_elements_by_class_name('isv-r.PNCib.MSM1fd')) < numImagesDesired:
        print(len(divContainingImages.find_elements_by_class_name('isv-r.PNCib.MSM1fd')))
        scroll_to_bottom() 

    boxesContainingImages = divContainingImages.find_elements_by_class_name('isv-r.PNCib.MSM1fd')
    print(len(boxesContainingImages)) 

    # loop through the boxes until our count is greater than numImagesDesired and we're done 
    for box in boxesContainingImages:
        count += 1
        if count > numImagesDesired:
            return 
        box.click() 
        time.sleep(random.randint(3,6))
        # give the http a certain amount of seconds to load in for the extension, if it doesn't load in 
        # this time, then we assume its a base64 encoded image and continue 
        
        # need a lot of error handling code here because there is a lot of uncertainity with the objects 
        # we're dealing with - IE we don't know how long the extension will take to get the real URL of the
        # image, we don't know the form of the data we return, whether its valid at all (not http/not base64) 

        actualImage = ""
        # if try succeeds, means we successfuly extracted a url 
        try:
            WebDriverWait(wd, 31).until(anchor_tag_has_http('ZsbmCf.vi_ext_addon'))
            actualImage = get_actual_image(wd)
        # otherwise, its a base64 encoded image 
        except:
            actualImage = get_actual_image(wd)
        finally:
            print(count)
            box.click()
            if "http" in actualImage:
                write_url_to_img(actualImage, count, query)
            else:
                try:
                    write_base64_to_img(actualImage,count, query)
                except:
                    continue 

In [316]:
def write_url_to_img(actualImage, imgNum, query):
    image_data = get_image_data_url(actualImage)
    img_format = "jpg" if ("jpg" in actualImage) else "png"
    write_to_img(image_data, imgNum, img_format, query) 

In [317]:
def get_actual_image(wd):
    view_image_box = lambda: wd.find_element_by_class_name("ZsbmCf.vi_ext_addon") 
    try:
        actualImage = view_image_box().get_attribute('href')
        return actualImage
    except:
        actualImage = view_image_box().get_attribute('href')
        return actualImage
    finally:
        print(actualImage[:5])

In [318]:
def get_image_data_url(url):
    image = requests.get(url)
    return image.content 

In [319]:
def get_image_path(imgNum, imgFormat, query):
    folderLoc = "/" + query
    imageName = "/" + query + str(imgNum)
    fileExt = "."+imgFormat
    return os.path.abspath('') + folderLoc + imageName + fileExt 

In [320]:
def write_to_img(imageBytesData, imgNum, imgFormat, query, url = True):
    imagePath = get_image_path(imgNum, imgFormat, query)
    if url:
        with open(imagePath, "wb") as f:
            f.write(imageBytesData)
    else:
        with open(imagePath, "wb") as f:
            f.write(base64.b64decode(imageBytesData))

In [321]:
def write_base64_to_img(encodedString, imgNum, query):
    img_src = encodedString.split(',')
    img_format = "jpg" if ("jpeg" in img_src[0]) else "png"
    write_to_img(img_src[1], imgNum, img_format, query, False)

In [322]:
image_scraper(["batman", "spiderman", "hulk", "superman"], 11, DRIVER_PATH, extension_path)

48
https
1
data:
2
https
3
https
4
https
5
https
6
https
7
https
8
https
9
https
10
https
11
48
https
1
https
2
https
3
https
4
https
5
https
6
https
7
https
8
https
9
https
10
https
11
48
https
1
https
2
https
3
https
4
https
5
https
6
https
7
https
8
https
9
https
10
https
11
48
https
1
https
2
https
3
https
4
https
5
https
6
https
7
https
8
https
9
https
10
https
11
